# Playing around with mixture of experts

TODO for 8/9: Train mixture of experts using SGD or EM. Read more about mixture of experts, boosting, and bayesian model averaging

## 1. Hyperparams, Imports, Utils

In [175]:
import plotly.graph_objs as go
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch import optim

In [176]:
N_POINTS = 1000
TEST_SIZE = 0.2

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

PLOT_COLORS = [
    "#1f77b4", # muted blue
    "#ff7f0e", # safety orange
    "#2ca02c", # cooked asparagus green
    "#d62728", # brick red
    "#9467bd", # muted purple
    "#8c564b", # chestnut brown
    "#e377c2", # raspberry yogurt pink
    "#7f7f7f", # middle gray
    "#bcbd22", # curry yellow-green
    "#17becf" # blue-teal
]

### 1.1 Plotting Utils

In [177]:
def scatter_plot(x, y, title="Scatter plot", xaxis_title="x", yaxis_title="y"):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=y, mode="markers", marker_color="purple"))
    fig.update_layout(title=title, xaxis_title=xaxis_title, yaxis_title=yaxis_title)
    fig.show()

def scatter_plot_with_line(x_scatter, y_scatter, x_lines, y_lines, title="Scatter plot", xaxis_title="x", yaxis_title="y"):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x_scatter, y=y_scatter, mode="markers", name="Train Data", marker_color="purple"))
    for i, (x_line, y_line) in enumerate(zip(x_lines, y_lines)):
        fig.add_trace(go.Scatter(x=x_line, y=y_line, mode="lines+markers", name=f"Fit {i}", marker_color=PLOT_COLORS[i]))
    fig.update_layout(title=title, xaxis_title=xaxis_title, yaxis_title=yaxis_title)
    fig.show()

def line_plot_with_markers(x_lines, y_lines, title="Line plot", xaxis_title="x", yaxis_title="y"):
    fig = go.Figure()
    for i, (x_line, y_line) in enumerate(zip(x_lines, y_lines)):
        fig.add_trace(go.Scatter(x=x_line, y=y_line, mode="lines+markers", name=f"Points {i}", marker_color=PLOT_COLORS[i]))
    fig.update_layout(title=title, xaxis_title=xaxis_title, yaxis_title=yaxis_title)
    fig.show()

## 2. Generate Datasets

Data:
-  https://scikit-learn.org/stable/datasets/sample_generators.html
- https://scikit-learn.org/stable/datasets/toy_dataset.html

### 2.1 Roof Dataset

In [178]:
def create_roof_data(noise=0.05):
    x_left = np.random.uniform(-2, -0.5, size=(N_POINTS//3))
    x_center = np.random.uniform(-0.75, 0.75, size=(N_POINTS//3))
    x_right = np.random.uniform(0.5, 2, size=(N_POINTS//3))

    y_left = x_left + 1.5 + np.random.normal(0, noise, size=x_left.shape)
    y_center = np.random.normal(0, noise, size=x_center.shape)
    y_right = -x_right + 1.5 + np.random.normal(0, noise, size=x_right.shape)

    x = np.concatenate([x_left, x_center, x_right])
    y = np.concatenate([y_left, y_center, y_right])

    idx = np.arange(x.shape[0])
    np.random.shuffle(idx)
    x = x[idx]
    y = y[idx]

    return x, y

x_roof, y_roof = create_roof_data()
scatter_plot(x_roof, y_roof, "Roof Data")

### 2.2 Zag Dataset

In [179]:
def create_zig_data(overlap=0, noise=0.05):
    x_left = np.random.uniform(-2 + overlap/2, 0 + overlap/2, size=(N_POINTS//2))
    x_right = np.random.uniform(0 - overlap/2, 2 - overlap/2, size=(N_POINTS//2))

    y_left = x_left + 0.5 + np.random.normal(0, noise, size=x_left.shape)
    y_right = x_right - 0.5 + np.random.normal(0, noise, size=x_right.shape)

    x = np.concatenate([x_left, x_right])
    y = np.concatenate([y_left, y_right])

    idx = np.arange(x.shape[0])
    np.random.shuffle(idx)
    x = x[idx]
    y = y[idx]

    return x, y

x_zig_over, y_zig_over = create_zig_data(overlap=0.5)
scatter_plot(x_zig_over, y_zig_over, "Zig Overlap Data")

x_zig_no_over, y_zig_no_over = create_zig_data(overlap=-0.5)
# x_zig_no_over, y_zig_no_over = create_zig_data(overlap=-2)
scatter_plot(x_zig_no_over, y_zig_no_over, "Zig No Overlap Data")

### 2.3 Linear Dataset

In [180]:
def create_linear_data(noise=0.05):
    x = np.random.uniform(-2, 2, size=(N_POINTS))
    y = x + np.random.normal(0, noise, size=x.shape)
    
    idx = np.arange(x.shape[0])
    np.random.shuffle(idx)
    x = x[idx]
    y = y[idx]

    return x, y

x_linear, y_linear = create_linear_data()
scatter_plot(x_linear, y_linear, "Linear Data")

### 2.4 Test Train Splits

In [181]:
x_roof_train, x_roof_test, y_roof_train, y_roof_test = train_test_split(x_roof, y_roof, test_size=TEST_SIZE)
x_zig_over_train, x_zig_over_test, y_zig_over_train, y_zig_over_test = train_test_split(x_zig_over, y_zig_over, test_size=TEST_SIZE)
x_zig_no_over_train, x_zig_no_over_test, y_zig_no_over_train, y_zig_no_over_test = train_test_split(x_zig_no_over, y_zig_no_over, test_size=TEST_SIZE)
x_linear_train, x_linear_test, y_linear_train, y_linear_test = train_test_split(x_linear, y_linear, test_size=TEST_SIZE)

## 3. Training

In [182]:
def train_and_plot_all_data(model):
    train_and_plot(model, x_roof_train, x_roof_test, y_roof_train, y_roof_test)
    train_and_plot(model, x_zig_over_train, x_zig_over_test, y_zig_over_train, y_zig_over_test)
    train_and_plot(model, x_zig_no_over_train, x_zig_no_over_test, y_zig_no_over_train, y_zig_no_over_test)
    train_and_plot(model, x_linear_train, x_linear_test, y_linear_train, y_linear_test)

def train_and_plot(model, x_train, x_test, y_train, y_test):
    model.fit(x_train, y_train)
    predictions = model.predict(x_test)

    mse = mean_squared_error(predictions, y_test)
    rounded_mse = round(mse, 6)

    x_plot, y_plot = model.plot()

    scatter_plot_with_line(x_train, y_train, x_plot, y_plot, title=f"{model.name} with Test MSE: {rounded_mse}")

    return predictions

## 4. Models

In [183]:
class LinearRegressor:
    def __init__(self):
        self.model = LinearRegression()
        self.name = "Linear Regression"
        
    def fit(self, x, y):
        self.model.fit(np.expand_dims(x, axis=1), y)
        
    def predict(self, x):
        return self.model.predict(np.expand_dims(x, axis=1))

    def plot(self):
        x = np.array([-2, 2])
        y = self.model.predict(np.expand_dims(x, axis=1))
        return [x], [y] 
    
    def __str__(self):
        return self.name

train_and_plot_all_data(LinearRegressor())

In [184]:
class BootstrapLinearRegressor:
    def __init__(self, n_regressors=3):
        self.models = [LinearRegression() for _ in range(n_regressors)]
        self.name = f"Bootstrapped Linear Regression with {n_regressors} Regressors"
        self.n_regressors = n_regressors
        
    def fit(self, x, y):
        for i in range(self.n_regressors):
            idxs = np.random.randint(0, x.shape[0], x.shape[0])
            self.models[i].fit(np.expand_dims(x[idxs], axis=1), y[idxs])
        
    def predict(self, x):
        predictions_array = np.concatenate([np.expand_dims(self.models[i].predict(np.expand_dims(x, axis=1)), axis=1) for i in range(self.n_regressors)], axis=1)

        return np.mean(predictions_array, axis=1)

    def plot(self):
        xs = []
        ys = []
        for model in self.models:
            x = np.array([-2, 2])
            xs.append(x)
            ys.append(model.predict(np.expand_dims(x, axis=1))) 
        return xs, ys 
    
    def __str__(self):
        return self.name

train_and_plot_all_data(BootstrapLinearRegressor())

In [185]:
class MixtureOfLinearExpertsNotConditional:
    def __init__(self, n_experts=3, iterations=10):
        self.pi = np.random.standard_normal(size=(n_experts))
        self.w = np.random.standard_normal(size=(n_experts, 2))
        self.beta = np.random.standard_normal()
        self.name = f"Mixture of Linear Experts Not Conditional with {n_experts} Experts"
        self.n_experts = n_experts
        self.iterations = iterations
        
    def fit(self, x, y):
        # x_lift = np.concatenate([x, np.ones((x.shape[0], 1))], axis=1)
        x_lift = np.concatenate([np.expand_dims(x, axis=1), np.ones((x.shape[0], 1))], axis=1)
        
        for iteration in range(self.iterations):
            # print((x_lift @ self.w.T).shape)
            # print((np.exp(-self.beta**2 * (np.expand_dims(y, axis=1) - x_lift @ self.w.T)**2)).shape)
            # ===Expectation===
            gamma = (1/np.sqrt(2 *np.pi)) * self.beta * np.exp(-self.beta**2 * (np.expand_dims(y, axis=1) - x_lift @ self.w.T)**2) * self.pi
            gamma = gamma / np.expand_dims(np.sum(gamma, axis=1), axis=1)
            print(gamma)

            # ===Maximizaton===
            self.pi = np.mean(gamma, axis=0)
            self.w = np.sum(gamma * x, axis=0) / np.sum(gamma, axis=0)
            self.beta = np.sum(gamma * (y - self.predict(x))**2) / np.sum(gamma)

            break

    def predict(self, x):
        return np.sum(self.pi * self.w * x, axis=1)

    def plot(self):
        xs = []
        ys = []
        for model in self.models:
            x = np.array([-2, 2])
            xs.append(x)
            ys.append(model.predict(np.expand_dims(x, axis=1))) 
        return xs, ys 
    
    def __str__(self):
        return self.name

# train_and_plot_all_data(MixtureOfLinearExpertsNotConditional())

In [189]:
class MixtureOfLinearExpertsConditionalModel(nn.Module):
    def __init__(self, n_experts=3):
        super().__init__()
        self.linear1 = nn.Linear(1, n_experts)
        self.linear2 = nn.Linear(1, n_experts)

    def forward(self, x):
        u = self.linear1(x)
        z = self.linear2(x)
        v = nn.Softmax(dim=1)(z)
        y = torch.sum(torch.mul(u, v), dim=1)
        return y
    
    def forward_weighting(self, x):
        z = self.linear2(x)
        v = nn.Softmax(dim=1)(z)
        return v


class LinearFunction():
    def __init__(self, m, b):
        self.m = m
        self.b = b
    
    def predict(self, x):
        return self.m * x + self.b
    

class MixtureOfLinearExpertsConditionalSGD():
    def __init__(self, n_experts=3, epochs=1000):
        self.model = MixtureOfLinearExpertsConditionalModel(n_experts=n_experts)
        self.name = f"Mixture of Linear Experts Conditional SGD with {n_experts} Experts"
        self.n_experts = n_experts
        self.epochs = epochs
        self.loss_fn = nn.MSELoss()
        self.optimizer = optim.Adam(self.model.parameters())
        
    def fit(self, x, y):
        losses = []
        for _ in range(self.epochs):
            self.optimizer.zero_grad()
            y_pred = self.model(torch.from_numpy(np.expand_dims(x, axis=1)).float())
            loss = self.loss_fn(y_pred, torch.tensor(y).float())
            losses.append(loss.item())
            loss.backward()
            self.optimizer.step()
        
        line_plot_with_markers([[i for i in range(len(losses))]], [losses], title="Loss Curve")
        
    def predict(self, x):
        return self.model(torch.from_numpy(np.expand_dims(x, axis=1)).float()).detach().numpy()

    def plot(self):
        params = {}
        for name, param in self.model.named_parameters():
            params[name] = np.ravel(param.data.numpy())

        linear_experts = []
        for i in range(self.n_experts):
            linear_experts.append(LinearFunction(params[f"linear1.weight"][i], params[f"linear1.bias"][i]))
        
        x_weight = np.linspace(-2, 2, 100)
        y_weight = self.model.forward_weighting(torch.from_numpy(np.expand_dims(x_weight, axis=1)).float()).detach().numpy()

        line_plot_with_markers([x_weight for _ in range(self.n_experts)], [y_weight[:, i] for i in range(self.n_experts)], title="Weighting Function")

        xs = []
        ys = []
        for linear_expert in linear_experts:
            x = np.array([-2, 2])
            xs.append(x)
            ys.append(linear_expert.predict(x))

        xs.append(np.linspace(-2, 2, 100))
        ys.append(self.model.forward(torch.from_numpy(np.expand_dims(np.linspace(-2, 2, 100), axis=1)).float()).detach().numpy())
        return xs, ys
    
    def __str__(self):
        return self.name

train_and_plot_all_data(MixtureOfLinearExpertsConditionalSGD())

In [187]:
class MixtureOfLinearExpertsConditional:
    def __init__(self, n_experts=3):
        pass

In [188]:
class MixtureOfLinearExpertsNotConditionalSGD:
    def __init__(self, n_experts=3):
        pass